# Monte Carlo battle simulations on generic graphs

In [2]:
import random
import itertools
import csv

# Initialize probability dictionary as PMF.
PROBABILITIES = {
    (1, 1): [(41.7, 'D loses 1 troop'), (58.3, 'A loses 1 troop')],
    (2, 1): [(57.9, 'D loses 1 troop'), (42.1, 'A loses 1 troop')],
    (3, 1): [(66.0, 'D loses 1 troop'), (34.0, 'A loses 1 troop')],
    (1, 2): [(25.5, 'D loses 1 troop'), (74.5, 'A loses 1 troop')],
    (2, 2): [(22.8, 'D loses 1 troop'), (44.8, 'A loses 2 troops'), (32.4, 'Both lose 1 troop')],
    (3, 2): [(37.2, 'D loses 2 troops'), (29.2, 'A loses 2 troops'), (33.6, 'Both lose 1 troop')]}

# Initialize graph adjacency matrices.
GRAPHS = {
    'M1': [[0, 1], [1, 0]],
    'M2': [[0, 1, 1], [1, 0, 0], [1, 0, 0]],
    'M3': [[0, 1, 1, 1], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0]],
    'M4': [[0, 1, 1, 1, 1], [1, 0, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 0, 0, 0]]}

def simulate_battle(attacking_troops, defending_troops):
    '''Simulates a battle and returns the result based on probabilities.'''
    outcome_probabilities = PROBABILITIES[(attacking_troops, defending_troops)]
    result = random.choices(outcome_probabilities, weights=[prob[0] for prob in outcome_probabilities])[0][1]
    return result

def perform_attack(node_a, node_b, node_ownership, node_troops):
    '''Performs an attack and updates the node ownership and troop counts.'''
    attacking_troops = min(node_troops[node_a] - 1, 3)
    defending_troops = min(node_troops[node_b], 2)

    while attacking_troops > 0 and defending_troops > 0:
        battle_result = simulate_battle(attacking_troops, defending_troops)
        if battle_result == 'D loses 1 troop':
            defending_troops -= 1
        elif battle_result == 'A loses 1 troop':
            attacking_troops -= 1
        elif battle_result == 'A loses 2 troops':
            attacking_troops -= 2
        elif battle_result == 'D loses 2 troops':
            defending_troops -= 2
        elif battle_result == 'Both lose 1 troop':
            attacking_troops -= 1
            defending_troops -= 1

    if defending_troops <= 0:
        node_ownership[node_b] = 'Player1'
        node_troops[node_b] = 0
        node_troops[node_a] -= (min(node_troops[node_a] - 1, 3) - attacking_troops)
    else:
        node_troops[node_a] -= (min(node_troops[node_a] - 1, 3) - attacking_troops)
        node_troops[node_b] = defending_troops

    return node_ownership, node_troops

def combine_ownership_and_troops(ownership, troops):
    '''Combines ownership and troops into a dictionary format.'''
    return {node: [1 if ownership[node] == 'Player1' else 2, troops[node]] for node in ownership}

def compute_new_state(final_state):
    '''Compute the 'new_state' based on the final_state.'''
    new_state = {key: value[:] for key, value in final_state.items()}

    if final_state[0][0] == 1 and final_state[0][1] == 0:
        max_node = max(
            (node for node, (owner, troops) in final_state.items() if owner == 1 and node != 0),
            key=lambda x: final_state[x][1], default=None)
        
        if max_node is not None:
            max_troops = final_state[max_node][1]
            new_state[0][1] += max_troops - 1
            new_state[max_node][1] = 1

    return new_state

def simulate_game(adjacency_matrix, node_ownership, node_troops, simulations=100):
    '''Simulates the game and returns the results of the simulation.'''
    results = []
    for _ in range(simulations):
        current_ownership = node_ownership.copy()
        current_troops = node_troops.copy()
        successful = False

        while True:
            attacking_node = max(
                (node for node, owner in current_ownership.items() if owner == 'Player1' and current_troops[node] > 1),
                key=lambda x: current_troops[x], default=None)
            
            if attacking_node is None:
                break
            adjacent_nodes = [i for i, is_adjacent in enumerate(adjacency_matrix[attacking_node]) if is_adjacent == 1]
            target_node = next((node for node in adjacent_nodes if current_ownership[node] != 'Player1'), None)
            
            if target_node is None:
                break
            current_ownership, current_troops = perform_attack(attacking_node, target_node, current_ownership, current_troops)
            
            if current_ownership.get(0) == 'Player1':
                successful = True

        initial_combined = combine_ownership_and_troops(node_ownership, node_troops)
        final_combined = combine_ownership_and_troops(current_ownership, current_troops)
        new_combined = compute_new_state(final_combined)

        results.append({
            'initial_state': initial_combined,
            'final_state': final_combined,
            'new_state': new_combined,
            'successful': 1 if successful else 0})

    return results

def save_results_to_csv(results, filename):
    '''Saves the simulation results to a CSV file.'''
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Matrix', 'Initial State', 'Final State', 'New State', 'Successful'])
        for result in results:
            writer.writerow([
                result['matrix'],
                result['initial_state'],
                result['final_state'],
                result['new_state'],
                result['successful']])

def run_simulation(matrix_choice, simulations):
    '''Runs the simulation and saves the results to a CSV file.'''
    adjacency_matrix = GRAPHS[matrix_choice]
    num_nodes = len(adjacency_matrix)

    results = []
    for troop_combination in itertools.product(range(1, 5), repeat=num_nodes):
        node_ownership = {i: 'Player1' if i != 0 else 'Player2' for i in range(num_nodes)}
        node_troops = {i: troop_combination[i] for i in range(num_nodes)}
        simulation_results = simulate_game(adjacency_matrix, node_ownership, node_troops, simulations)
        for result in simulation_results:
            result['matrix'] = matrix_choice
        results.extend(simulation_results)

    save_results_to_csv(results, f'{matrix_choice}_simulation_results.csv')
    print(f'Battle simulations on graph {matrix_choice} has been saved to "{matrix_choice}_simulation_results.csv"')

# Simulate 1000 battles on graphs M1-M3.
def generate_battle_data(k=3):
    for i in range(1, k+1):
        num_simulatons = 1000
        graph_size = f'M{i}'
        run_simulation(graph_size, num_simulatons)

In [3]:
#generate_battle_data()

Results saved to M1_simulation_results.csv
Results saved to M2_simulation_results.csv
Results saved to M3_simulation_results.csv
